# Description:
This Jupyter notebook contains all python functions developed for the galactic noise study in order
to allow for easy importing. The common functions are organised per notebook in which they were first used.

# Import modules:

In [4]:
import numpy as np
import scipy as sc
import scipy.fft as scfft
import matplotlib.pyplot as plt
import uproot
import os
import scipy.integrate
from scipy.optimize import curve_fit
#%matplotlib widget

# Define Constants:

# Galaxy Stacking:

In [1]:
from datetime import datetime

from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy import units as u

In [2]:
def ADCtoVoltageTemp(ADCCounts):
    ADC_Factor=0.618
    ADC_Offset=-8.133 
    return (ADC_Factor*ADCCounts + ADC_Offset)

In [3]:
def LST(TriggerTimes,EvIndex):
    """Computes the Local Sidereal Time in decimal hours via the astropy module"""
    observing_location = EarthLocation(lat=72.598265*u.deg, lon=-38.459936*u.deg)
    observing_time = Time(datetime.utcfromtimestamp(TriggerTimes[EvIndex]), scale='utc', location=observing_location)
    T=observing_time.sidereal_time('mean')
    return T.hour

In [4]:
def UTC(TriggerTimes,EvIndex):
    """Computes the UTC time in decimal hours"""
    T=datetime.utcfromtimestamp(TriggerTimes[EvIndex])
    return T.hour+T.minute/60 + T.second/3600

In [2]:
def DailyVoltAvg(StNr,ChNr,Runs,NBins=24,ZeroAvg=True):
    NRuns=0
    EventRMS=np.array([]) #Array to store V_RMS value of each event
    EventTime=np.array([])#Array to store timestamp of each event
    for Run in Runs:
        path=Path(StNr,Run)
        #if os.path.isdir(path+"/combined.root"):
        if os.path.isfile(path+"/combined.root") and os.path.isfile(path+"/daqstatus.root"):
            NRuns+=1
        
            CombinedFile, DAQStatFile, HeadersFile, PedestalFile=FilesStRun(StNr,Run)
            RadiantData=CombinedFile['combined']['waveforms']['radiant_data[24][2048]'].array(library='np')
            EventNrs=CombinedFile['combined']['waveforms']['event_number'].array(library="np")
            TriggerTimes=CombinedFile['combined']['header']["trigger_time"].array(library='np')        
                
            for EvNr in EventNrs:
                EvIndex=np.where(EventNrs==EvNr)[0][0]
                if ZeroAvg==True:
                    Vmean=np.mean(ADCtoVoltage(RadiantData[EvIndex][ChNr]))
                    EventRMS=np.append(EventRMS,np.sqrt(np.mean([(V-Vmean)**2 for V in ADCtoVoltage(RadiantData[EvIndex][ChNr])])))
                else:
                    EventRMS=np.append(EventRMS,np.sqrt(np.mean([V**2 for V in ADCtoVoltage(RadiantData[EvIndex][ChNr])])))
                EventTime=np.append(EventTime,LST(TriggerTimes,EvIndex))
                
                
    #print(np.sum([EventRMS[i] for i in np.arange(len(EventTime)) if EventTime[i]<=.25 ]))
    
    EventTimeCounts, EventTimeBins=np.histogram(EventTime, bins=NBins,range=(0,24),density=False) #Storing timestamps in histogram format
    #Make a histogram of the V_RMS value fully added in its respective bin by adding V_RMS as a weigth to the additions to the histogram
    EventRMSCounts, EventRMSBins=np.histogram(EventTime, bins=NBins,range=(0,24),density=False,weights=EventRMS)    
    
    ##plt.hist(EventTime, bins=NBins,range=(0,24),density=False)
    
    ##plt.figure()
    ##plt.hist(EventTime, bins=NBins,range=(0,24),density=False, weights=EventRMS)
    
    #RMSBins=np.digitize(EventTime,EventTimeBins) #Array of idx of bin in which the timestamp of each event falls
    
    ##Make a histogram of the V_RMS value fully added in its respective bin by adding V_RMS as a weigth to the additions to the histogram
    #EventRMSCounts, EventRMSBins=np.histogram(RMSBins, bins=24,range=(0,24),density=False,weights=EventRMS)
    

    
    MidBins= np.array([(EventTimeBins[i] + EventTimeBins[i+1])/2 for i in range(0,len(EventTimeBins)-1)])           
    VRMSAvg=np.array([EventRMSCounts[i]/EventTimeCounts[i]  if EventTimeCounts[i] !=0 else 0 for i in range(len(EventRMSCounts))])
    
    plt.figure(figsize=(15,5))
    plt.figtext(0.2, 0.8, "Entries:" + str(np.sum(EventTimeCounts)), fontsize=18,bbox=dict(edgecolor='black', facecolor='none', alpha=0.2, pad=10.0))
    #plt.plot(SamplingTimes*(10**9),TimeTrace,'-')#, label="Channel " + str(ChNr))
    #plt.plot(Energies,TritonEnergyLoss,'-',color='r', label="Triton")
    #plt.hist(RMSBins, bins=24,range=(0,24),density=False, weights=[EventRMS[i]/EventRMSCounts[i] for i in range(len(EventRMS))])
    plt.plot(MidBins,1000*VRMSAvg,'.')
    plt.grid(color='grey', linestyle='-', linewidth=1,alpha=0.5)
    plt.title("V_RMS of Station " + str(StNr) + ", channel " + str(ChNr) + " for " + str(NRuns) + " runs between run " + str(Runs[0]) + " and run " + str(Runs[-1]) + " throughout the day for " + str(NBins) + " bins")
    #plt.ylim(-50,50)
    #plt.xlim(0,np.max(SamplingTimes*(10**9)))
    plt.xlabel("Time (hrs)",fontsize=20)#20)
    plt.ylabel("V_RMS (mV)",fontsize=20)#20)
    plt.xticks(np.arange(0, 24, 1.0),fontsize=25)#15)
    plt.yticks(fontsize=25)#15)
    #plt.legend()
    plt.show()

In [5]:
def VRMSTimeOfDay(StNr,ChNr,Runs,t0,t1):
    EventRMS=np.array([]) #Array to store V_RMS value of each event
    EventTime=np.array([])#Array to store timestamp of each event
    for Run in Runs:
        path=Path(StNr,Run)
        #if os.path.isdir(path+"/combined.root"):
        if os.path.isfile(path+"/combined.root"):

            CombinedFile, DAQStatFile, HeadersFile, PedestalFile=FilesStRun(StNr,Run)
            RadiantData=CombinedFile['combined']['waveforms']['radiant_data[24][2048]'].array(library='np')
            EventNrs=CombinedFile['combined']['waveforms']['event_number'].array(library="np")
            TriggerTimes=CombinedFile['combined']['header']["trigger_time"].array(library='np')        
            for EvNr in EventNrs:
                EvIndex=np.where(EventNrs==EvNr)[0][0]
                if t0<=LST(TriggerTimes,EvIndex)<=t1:
                    EventRMS=np.append(EventRMS,np.sqrt(np.mean([V**2 for V in ADCtoVoltageTemp(RadiantData[EvIndex][ChNr])])))

    
    plt.plot(np.arange(len(EventRMS)),EventRMS,".")
    plt.xlabel("Sample Nr")
    plt.ylabel(r"$V_{RMS}$")
    plt.show()

In [6]:
def EvntsPerRun(StNr,ChNr,Runs):
    NRuns=0
    AmountEvents=np.array([])
    ValidRuns=np.array([])
    for Run in Runs:
        path=Path(StNr,Run)
        #if os.path.isdir(path+"/combined.root"):
        if os.path.isfile(path+"/combined.root") and os.path.isfile(path+"/daqstatus.root"):
            NRuns+=1
            
            CombinedFile, DAQStatFile, HeadersFile, PedestalFile=FilesStRun(StNr,Run)
            RadiantData=CombinedFile['combined']['waveforms']['radiant_data[24][2048]'].array(library='np')
            EventNrs=CombinedFile['combined']['waveforms']['event_number'].array(library="np")
            #TriggerTimes=CombinedFile['combined']['header']["trigger_time"].array(library='np')        
            AmountEvents=np.append(AmountEvents,len(EventNrs))  
            ValidRuns=np.append(ValidRuns,Run)  
            
    plt.figure()
    plt.plot(ValidRuns,AmountEvents,".")
    plt.figtext(0.2, 0.8, "Entries:" + str(np.sum(AmountEvents)), fontsize=18,bbox=dict(edgecolor='black', facecolor='none', alpha=0.2, pad=10.0))
    plt.xlabel("Run Nr")
    plt.ylabel("Amount of events")
    plt.title("Amount of events per run for Station " + str(StNr) + ", channel " + str(ChNr))
    plt.yscale("log")
    plt.show()

# Investigating Transit Curves:

In [4]:
def TransitCurves(StNr,ExtraChNr,Runs,NBins=24,ZeroAvg=True):
    """Plots the Vrms values as a function of LST for the upwards facing LPDA's and a chosen channel."""
    ChNrs=[13,16,19,ExtraChNr]
    NRuns=0
    EventRMS= np.empty((4,0),float) #Array to store V_RMS value of each event for all four channels
    EventTime=np.array([])#Array to store timestamp of each event
    for Run in Runs:
        path=Path(StNr,Run)
        #if os.path.isdir(path+"/combined.root"):
        if os.path.isfile(path+"/combined.root") and os.path.isfile(path+"/daqstatus.root"):
            NRuns+=1
        
            CombinedFile, DAQStatFile, HeadersFile, PedestalFile=FilesStRun(StNr,Run)
            RadiantData=CombinedFile['combined']['waveforms']['radiant_data[24][2048]'].array(library='np')
            EventNrs=CombinedFile['combined']['waveforms']['event_number'].array(library="np")
            TriggerTimes=CombinedFile['combined']['header']["trigger_time"].array(library='np')        
                
            for EvNr in EventNrs:
                EvIndex=np.where(EventNrs==EvNr)[0][0]
                #EventRMS=np.append(EventRMS,np.sqrt(np.mean([V**2 for V in ADCtoVoltage(RadiantData[EvIndex][ChNr])])))
                if ZeroAvg==True:
                    VmeanList=[np.mean(ADCtoVoltage(RadiantData[EvIndex][ChNr])) for ChNr in ChNrs]
                    VRMSList=[[np.sqrt(np.mean([(V-VmeanList[i])**2 for V in ADCtoVoltage(RadiantData[EvIndex][ChNrs[i]])]))] for i in range(len(ChNrs))]
                else:
                    VRMSList=[[np.sqrt(np.mean([V**2 for V in ADCtoVoltage(RadiantData[EvIndex][ChNr])]))] for ChNr in ChNrs ]
                EventRMS=np.concatenate((EventRMS,np.array(VRMSList)),axis=1)
                EventTime=np.append(EventTime,LST(TriggerTimes,EvIndex))
                
                
    #print(np.sum([EventRMS[i] for i in np.arange(len(EventTime)) if EventTime[i]<=.25 ]))
    
    EventTimeCounts, EventTimeBins=np.histogram(EventTime, bins=NBins,range=(0,24),density=False) #Storing timestamps in histogram format
    #Make a histogram of the V_RMS value fully added in its respective bin by adding V_RMS as a weigth to the additions to the histogram
    
    EventRMSCountsList=np.array([np.histogram(EventTime, bins=NBins,range=(0,24),density=False,weights=EventRMS[ChIdx]) for ChIdx in np.arange(4)],dtype=object)
    #Indexable EventRMSCountsList[ChNr][0 for Counts, 1 for Bins][BinIdx]
    
    #RMSBins=np.digitize(EventTime,EventTimeBins) #Array of idx of bin in which the timestamp of each event falls
    
    ##Make a histogram of the V_RMS value fully added in its respective bin by adding V_RMS as a weigth to the additions to the histogram
    #EventRMSCounts, EventRMSBins=np.histogram(RMSBins, bins=24,range=(0,24),density=False,weights=EventRMS)
    

    
    MidBins= np.array([(EventTimeBins[i] + EventTimeBins[i+1])/2 for i in range(0,len(EventTimeBins)-1)])           
    VRMSAvg=np.array([[EventRMSCountsList[ChIdx][0][i]/EventTimeCounts[i]  if EventTimeCounts[i] !=0 else 0 for i in range(len(EventTimeCounts))] for ChIdx in np.arange(4)])
    #Indexabl as VRMSAvg[ChNr][BinIdx]
    if False:
        plt.figure(figsize=(15,5))
        plt.figtext(0.2, 0.8, "Entries:" + str(np.sum(EventTimeCounts)), fontsize=18,bbox=dict(edgecolor='black', facecolor='none', alpha=0.2, pad=10.0))
        #plt.plot(SamplingTimes*(10**9),TimeTrace,'-')#, label="Channel " + str(ChNr))
        #plt.plot(Energies,TritonEnergyLoss,'-',color='r', label="Triton")
        #plt.hist(RMSBins, bins=24,range=(0,24),density=False, weights=[EventRMS[i]/EventRMSCounts[i] for i in range(len(EventRMS))])
        plt.plot(MidBins,VRMSAvg[0],'.')
        plt.grid(color='grey', linestyle='-', linewidth=1,alpha=0.5)
        plt.title("V_RMS of Station " + str(StNr) + ", channel " + str(ChNrs[0]) + " for " + str(NRuns) + " runs between run " + str(Runs[0]) + " and run " + str(Runs[-1]) + " throughout the day for " + str(NBins) + " bins")
        #plt.ylim(-50,50)
        #plt.xlim(0,np.max(SamplingTimes*(10**9)))
        plt.xlabel("Time (hrs)",fontsize=20)#20)
        plt.ylabel("V_RMS (V)",fontsize=20)#20)
        plt.xticks(np.arange(0, 24, 1.0),fontsize=25)#15)
        plt.yticks(fontsize=25)#15)
        #plt.legend()
        plt.show()
    
    fig, axs = plt.subplots(2, 2, figsize=(20, 10))
    fig.suptitle("Transit curve for StNr " + str(StNr) + ", for " + str(np.sum(EventTimeCounts)) + " entries", fontsize=25)
    #fig.text(0.30, 0.87, r"Null hypothesis $H_0$: dice are functioning properly and follow a uniform distribution", fontsize=10)
    #plt.subplots_adjust(top=0.82)
    #fig.title(r"Null hypothesis $H_0$: dice are functioning properly and follow a uniform distribution")
    
    for i in np.arange(4):
        yidx=i%2
        if i <2:
            xidx=0
        else:
            xidx=1
        axs[xidx, yidx].plot(MidBins,1000*VRMSAvg[i],'.')
        axs[xidx, yidx].grid(color='grey', linestyle='-', linewidth=1,alpha=0.5)
        #axs[0, 0].legend()
        axs[xidx, yidx].set_title("Channel " + str(ChNrs[i]), fontsize=20)
        if xidx==1:
            axs[xidx, yidx].set_xlabel("LST",fontsize=19)
        if yidx==0:
            axs[xidx, yidx].set_ylabel(r"$V_{RMS}$ in mV",fontsize=19)
        axs[xidx, yidx].set_xticks(np.arange(0, 24, 1.0))
        axs[xidx, yidx].xaxis.set_tick_params(labelsize=18)
        axs[xidx, yidx].yaxis.set_tick_params(labelsize=18)
        #axs[xidx, yidx].set_yticks(fontsize=25)

    fig.tight_layout()
    #fig.subplots_adjust(hspace=0.4)
    #plt.figtext(0.25, 0.01, "The analysis indicates a " + str(np.round(2*100*PVal,4)) + r"% chance that $H_0$ is true", fontsize=18)
    #plt.text(200,-200,"The analysis indicates a " + str(2*100*PVal) + "% chance that H_0 is true")
    plt.show()

In [1]:
def RunStartTime(StNr, Run):
    """Returns the start time of a run in Unix time"""
    CombinedFile, DAQStatFile, HeadersFile, PedestalFile=FilesStRun(StNr,Run)
    TriggerTimes=CombinedFile['combined']['header']["trigger_time"].array(library='np') 
    #print(datetime.utcfromtimestamp(TriggerTimes[0]))
    return TriggerTimes[0]

In [2]:
def TimedRunList(OGStNr,StNr,RunsRange=[0,100]):
    """Highly specific function that returns runs of a station StNr that occured during runs RunsRange of another station OGStNr"""
    RunTimes=[RunStartTime(OGStNr, RunsRange[0]),RunStartTime(OGStNr, RunsRange[1])]
    RunList=np.array([])
    for Run in np.arange(0,1000,dtype=int):
        path=Path(StNr,Run)
        if os.path.isfile(path+"/combined.root") and os.path.isfile(path+"/daqstatus.root"):
            if RunTimes[0] < RunStartTime(StNr, Run) < RunTimes[1]:
                RunList=np.append(RunList,Run)
    return RunList

In [5]:
def RunsInTimeframe(StNr,t0,t1,Runs=np.arange(1000)):
    """Returns a list of all runs of this station which are within the given LST"""
    RunList=np.array([]) #Array to store run values
    EventList=[]
    for Run in Runs:
        path=Path(StNr,Run)
        #if os.path.isdir(path+"/combined.root"):
        if os.path.isfile(path+"/combined.root"):

            CombinedFile, DAQStatFile, HeadersFile, PedestalFile=FilesStRun(StNr,Run)
            RadiantData=CombinedFile['combined']['waveforms']['radiant_data[24][2048]'].array(library='np')
            EventNrs=CombinedFile['combined']['waveforms']['event_number'].array(library="np")
            TriggerTimes=CombinedFile['combined']['header']["trigger_time"].array(library='np') 
            for EvNr in EventNrs:
                EvIndex=np.where(EventNrs==EvNr)[0][0]
                if t0<=LST(TriggerTimes,EvIndex)<=t1:
                    if not Run in RunList:
                        RunList=np.append(RunList,int(Run))
                        EventList.append([EvNr])
                    else:
                        EventList[-1].append(EvNr)  

    return RunList, EventList